# Model Server

In [1]:
# nuclio: ignore
import nuclio

In [18]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel
%nuclio config spec.build.baseImage = "mlrun/mlrun:0.4.8"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun:0.4.8'


In [19]:
%%nuclio cmd -c
python -m pip install numpy cloudpickle v3io sklearn

In [20]:
import os
from cloudpickle import load
import numpy as np
from typing import List
from datetime import datetime
import mlrun

In [21]:
class ClassifierModel(mlrun.runtimes.MLModelServer):
    def load(self):
        """Load model from storage."""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.model.predict(feats)
            resp = result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")
        
        return resp

In [22]:
# nuclio: end-code

# Test models locally and deploy

The sklearn-project generated one or more models that will be deployed in the server project `sklearn-servers`

### test locally

In [27]:
import cloudpickle as cp
from os import path
from mlrun import mlconf
models_path = path.join(mlconf.artifact_path, 'sklearn/sklearn.ensemble.RandomForestClassifier')

from sklearn.datasets import load_iris
iris = load_iris()

x = iris['data'].tolist()
y = iris['target']

for model in os.listdir(models_path):
    if model.endswith(".pkl"):
        
        my_server = ClassifierModel('classifier', model_dir=os.path.join(models_path, model))
        my_server.load()

        a = my_server.predict({"instances": x})

        assert len(a)==150

## document and save

In [28]:
from mlrun import new_model_server
fn = new_model_server('sklearn-server', model_class='ClassifierModel')
fn.spec.description = "generic sklearn model server"
fn.metadata.categories = ['serving', 'ml']
fn.metadata.labels = {'author': 'yaronh', 'framework': 'sklearn'}
#print(fn.to_yaml())
fn.export()

[mlrun] 2020-05-25 03:24:50,705 function spec saved to path: function.yaml


##  deploy server

In [29]:
from mlutils import get_vol_mount
fn.apply(get_vol_mount())
fn.set_envs({'SERVING_MODEL_iris_dataset_v1': models_path,
             'INFERENCE_STREAM': 'users/admin/tststream'})
#fn.verbose = True
fn.deploy(project='sk-project')

[mlrun] 2020-05-25 03:24:52,833 deploy started
[nuclio] 2020-05-25 03:25:02,973 (info) Build complete
[nuclio] 2020-05-25 03:26:02,935 (error) Failed to deploy. Details:

Error - context deadline exceeded
    .../platform/kube/controller/nucliofunction.go:134

Call stack:
Failed to wait for function resources to be available
    .../platform/kube/controller/nucliofunction.go:134



DeployError: cannot deploy 